In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

import prepare 
import acquire  

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import seaborn as sns
import matplotlib.pyplot as plt
from mergedeep import merge

In [2]:
#Acquire the URLS for the Repositories to explore

headers = acquire.headers
endpoint = "https://api.github.com/search/repositories"
query = "customer in:name"
sort = "stars"
per_page = 100
order = "desc"
page = 1

##Make the request
response = requests.get(f"{endpoint}?q={query}&sort={sort}&per_page={per_page}&order={order}&page={page}", headers=headers)
payload = response.json()
#payload['items']

In [3]:
page = 2

##Make the request
response = requests.get(f"{endpoint}?q={query}&sort={sort}&per_page={per_page}&order={order}&page={page}", headers=headers)
payload2 = response.json()
merge(payload, payload2)
payload['items']

[{'id': 178226060,
  'node_id': 'MDEwOlJlcG9zaXRvcnkxNzgyMjYwNjA=',
  'name': 'E-commerce-Customer-service-robot',
  'full_name': 'gongdelu/E-commerce-Customer-service-robot',
  'private': False,
  'owner': {'login': 'gongdelu',
   'id': 37997947,
   'node_id': 'MDQ6VXNlcjM3OTk3OTQ3',
   'avatar_url': 'https://avatars.githubusercontent.com/u/37997947?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/gongdelu',
   'html_url': 'https://github.com/gongdelu',
   'followers_url': 'https://api.github.com/users/gongdelu/followers',
   'following_url': 'https://api.github.com/users/gongdelu/following{/other_user}',
   'gists_url': 'https://api.github.com/users/gongdelu/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/gongdelu/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/gongdelu/subscriptions',
   'organizations_url': 'https://api.github.com/users/gongdelu/orgs',
   'repos_url': 'https://api.github.com/users/gongdelu/repos',


In [4]:
page = 3

##Make the request
response = requests.get(f"{endpoint}?q={query}&sort={sort}&per_page={per_page}&order={order}&page={page}", headers=headers)
payload2 = response.json()
merge(payload, payload2)
payload['items']

[{'id': 118996371,
  'node_id': 'MDEwOlJlcG9zaXRvcnkxMTg5OTYzNzE=',
  'name': 'CustomerChurn',
  'full_name': 'smit5490/CustomerChurn',
  'private': False,
  'owner': {'login': 'smit5490',
   'id': 13723626,
   'node_id': 'MDQ6VXNlcjEzNzIzNjI2',
   'avatar_url': 'https://avatars.githubusercontent.com/u/13723626?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/smit5490',
   'html_url': 'https://github.com/smit5490',
   'followers_url': 'https://api.github.com/users/smit5490/followers',
   'following_url': 'https://api.github.com/users/smit5490/following{/other_user}',
   'gists_url': 'https://api.github.com/users/smit5490/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/smit5490/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/smit5490/subscriptions',
   'organizations_url': 'https://api.github.com/users/smit5490/orgs',
   'repos_url': 'https://api.github.com/users/smit5490/repos',
   'events_url': 'https://api.github.com

In [5]:
#Convert the reponse to a data frame
urls_df = pd.DataFrame(payload['items'])

In [6]:
#Check the shape of the dataframe
urls_df.shape

(100, 79)

In [7]:
urls_df

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,allow_forking,is_template,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score
0,118996371,MDEwOlJlcG9zaXRvcnkxMTg5OTYzNzE=,CustomerChurn,smit5490/CustomerChurn,False,"{'login': 'smit5490', 'id': 13723626, 'node_id...",https://github.com/smit5490/CustomerChurn,None,False,https://api.github.com/repos/smit5490/Customer...,...,True,False,[],public,8,4,10,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
1,58745539,MDEwOlJlcG9zaXRvcnk1ODc0NTUzOQ==,vc-module-customer,VirtoCommerce/vc-module-customer,False,"{'login': 'VirtoCommerce', 'id': 5762443, 'nod...",https://github.com/VirtoCommerce/vc-module-cus...,Contacts management module,False,https://api.github.com/repos/VirtoCommerce/vc-...,...,True,False,[],public,21,14,10,dev,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
2,62332503,MDEwOlJlcG9zaXRvcnk2MjMzMjUwMw==,customer_portal,SonarSoftware/customer_portal,False,"{'login': 'SonarSoftware', 'id': 12124408, 'no...",https://github.com/SonarSoftware/customer_portal,A customer portal for Sonar.,False,https://api.github.com/repos/SonarSoftware/cus...,...,True,False,[],public,15,1,10,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
3,85580569,MDEwOlJlcG9zaXRvcnk4NTU4MDU2OQ==,customer-portal,oroinc/customer-portal,False,"{'login': 'oroinc', 'id': 4165718, 'node_id': ...",https://github.com/oroinc/customer-portal,OroCommerce package with customer portal and n...,False,https://api.github.com/repos/oroinc/customer-p...,...,True,False,[],public,15,5,10,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
4,47967191,MDEwOlJlcG9zaXRvcnk0Nzk2NzE5MQ==,customer-scripts,microsoft/customer-scripts,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/customer-scripts,Repository of scripts provided frequently to o...,False,https://api.github.com/repos/microsoft/custome...,...,True,False,[],public,13,1,10,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,69983058,MDEwOlJlcG9zaXRvcnk2OTk4MzA1OA==,ruby-sms-customer-engagement,Nexmo/ruby-sms-customer-engagement,False,"{'login': 'Nexmo', 'id': 551057, 'node_id': 'M...",https://github.com/Nexmo/ruby-sms-customer-eng...,Two-way SMS for customer engagement with Ruby,False,https://api.github.com/repos/Nexmo/ruby-sms-cu...,...,True,False,[tutorial],public,3,0,7,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
96,181278845,MDEwOlJlcG9zaXRvcnkxODEyNzg4NDU=,Kaggle_santander-customer-transaction-prediction,Rajneesh-Tiwari/Kaggle_santander-customer-tran...,False,"{'login': 'Rajneesh-Tiwari', 'id': 16912628, '...",https://github.com/Rajneesh-Tiwari/Kaggle_sant...,Code for 118th position in Kaggle santander-cu...,False,https://api.github.com/repos/Rajneesh-Tiwari/K...,...,True,False,[],public,3,0,7,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
97,149399792,MDEwOlJlcG9zaXRvcnkxNDkzOTk3OTI=,Exploratory-Data-Analysis-Customer-Churn-Predi...,Shubha23/Exploratory-Data-Analysis-Customer-Ch...,False,"{'login': 'Shubha23', 'id': 17091320, 'node_id...",https://github.com/Shubha23/Exploratory-Data-A...,Application of K-means clustering. Prediction ...,False,https://api.github.com/repos/Shubha23/Explorat...,...,True,False,[],public,3,0,7,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
98,113375100,MDEwOlJlcG9zaXRvcnkxMTMzNzUxMDA=,woocommerce-customer-source,devinsays/woocommerce-customer-source,False,"{'login': 'devinsays', 'id': 431879, 'node_id'...",https://github.com/devinsays/woocommerce-custo...,Learn where your WooCommerce customers are com...,False,https://api.github.com/repos/devinsays/woocomm...,...,True,False,[],public,2,0,7,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0


In [8]:
#Get a list of the full_name for the urls. Saved to acquire seperate file as repos and assigned to a variable
urls_df.full_name.to_list()

['smit5490/CustomerChurn',
 'VirtoCommerce/vc-module-customer',
 'SonarSoftware/customer_portal',
 'oroinc/customer-portal',
 'microsoft/customer-scripts',
 'correooke/example-customers-app',
 'darkdreamingdan/mlnd-customer-segments',
 'emilybache/CustomerSync-Refactoring-Kata',
 'bestkao/customer_segments',
 'devmentors/Pacco.Services.Customers',
 'toshi-k/kaggle-santander-customer-satisfaction',
 'cometchat-pro-tutorials/react-customer-support-live-widget',
 'celestinhermez/sparkify_customer_churn',
 'Ifihan/customer-management-app',
 'skioo/django-customer-billing',
 'anqi666/Customer-Data-Analysis',
 'mutedblues/Customer-Success-Management',
 'ho-nl/magento1-Ho_Customer',
 'aryashah2k/Datalogy-Customer-Segmentation-Data-Science-Internship',
 'Horizon733/customer-care-chatbot',
 'dserradji/reactive-customer-service',
 'Ashniu123/nestjs-customer-order-eventsourcing-cqrs',
 'M-R-K-Development/bc-customer-plugin',
 'melihbodr/Sentiment_analysis_to_customer_comments',
 'nd009/creating_c

In [9]:
#Use the scrape github function from the acquire module to acquire the dataframe containing the names of the
#repositories and the contents of the readme files
df = acquire.scrape_github_data()

#convert the data into a dataframe
df = pd.DataFrame(df)
df.head()

,repo,language,readme_contents
0,gongdelu/E-commerce-Customer-service-robot,Jupyter Notebook,# E-commerce-Customer-service-robot\n电商多轮对话客服机...
1,Azure/fta-customerfacingapps,C#,# FastTrack for Azure\n\nSee our [FastTrack fo...
2,ELMAHDI-AR/CustomerAuthentication-With-ASP.NET...,PowerShell,
3,kpei/Customer-Analytics,Jupyter Notebook,# Customer Analytics iPython Notebook\nSee htt...
4,Sylius/CustomerOrderCancellationPlugin,PHP,> :warning: **BEWARE!**\n> This repository has...


In [10]:
# read in the json file generated by compiling the acquire.py file
df = pd.read_json('data2.json')

In [11]:
df.head()

,repo,language,readme_contents
0,gongdelu/E-commerce-Customer-service-robot,Jupyter Notebook,# E-commerce-Customer-service-robot\n电商多轮对话客服机...
1,Azure/fta-customerfacingapps,C#,# FastTrack for Azure\n\nSee our [FastTrack fo...
2,ELMAHDI-AR/CustomerAuthentication-With-ASP.NET...,PowerShell,
3,kpei/Customer-Analytics,Jupyter Notebook,# Customer Analytics iPython Notebook\nSee htt...
4,Sylius/CustomerOrderCancellationPlugin,PHP,> :warning: **BEWARE!**\n> This repository has...


In [14]:
def is_foreign(texts):
    '''
    This function takes in a dataframe and return true if the scanned text is i†gn chinese
    '''
    if re.search("[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f\u3131-\uD79D]", texts):
            return True

In [15]:
df = df[df.readme_contents.apply(is_foreign) !=True]

In [12]:
# do drop unneeded data first then prep
df = prepare.drop_unneeded_data(df)

In [16]:
len(df)

122

In [18]:
df.head(50)

,repo,language,readme_contents
0,kpei/Customer-Analytics,Jupyter Notebook,# Customer Analytics iPython Notebook\nSee htt...
1,Sylius/CustomerOrderCancellationPlugin,PHP,> :warning: **BEWARE!**\n> This repository has...
2,optiflow/rfm-customer-segmentation,Jupyter Notebook,"# Recency, Frequency, and Monetary (RFM) Custo..."
3,Sylius/CustomerReorderPlugin,PHP,> :warning: **BEWARE!**\n> This repository has...
4,Prakhar-FF13/Customer-Analytics,Jupyter Notebook,## Business Problem :\n\n#### Problem statemen...
5,navdeep-G/customer-churn,Jupyter Notebook,# Customer Churn Analysis\n\n## Overview\n - ...
6,php-cuong/magento2-customer-avatar,PHP,# Magento 2 Customer Avatar\nThis is an awesom...
7,printu/customerio,PHP,# Customer.io API Client\n\nPHP bindings for t...
8,kiwicommerce/magento2-login-as-customer,PHP,"# We're not maintaining this extension, if you..."
9,klein0r/magento-customer-helper,PHP,Magento Help Customers Extension\n============...


## Explore

In [128]:
def remove_urls(text):
    '''
    This function removes urls from the string 
    '''
    res = []
    for i in text.strip().split():
        if not re.search(r"(https?)", i):  
            res.append(re.sub(r"[^A-Za-z\.]", "", i).replace(".", " "))   
    return " ".join(map(str.strip, res))

In [141]:
def remove_unicode_text(words):
    words_clean = re.sub(r'\<[^>]*\>', '', words)
    
    return words_clean

In [110]:
def remove_stopwords(content):
    '''
    This function removes English language stop words from our texts.
    '''
    stopword_list = stopwords.words('english')

    stopword_list.remove('no')
    stopword_list.remove('not')
    
    words = content.split()
    filtered_words = [w for w in words if w not in stopword_list]
    
    content_without_stopwords = ' '.join(filtered_words)
    
    return content_without_stopwords

In [111]:
def clean(text):
    '''Simplified text cleaning function'''
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return re.sub(r"[^a-z0-9\s]", '', text)

In [133]:
words = ' '.join(df.readme_contents)

In [116]:
words_clean = clean(words)

In [117]:
words_clean[0:1000]

' customer analytics ipython notebook\nsee httpkpeimeblogp921\n  warning beware\n this repository has been deprecated and will not be maintained or evolved by the sylius team you can still use it with compatible sylius versions but at your own risk as no bugs will be fixed on it\n\np aligncenter\n    a hrefhttpssyliuscom targetblank\n        img srchttpsdemosyliuscomassetsshopimglogopng \n    a\np\n\nh1 aligncentercustomer order cancellation pluginh1\n\np aligncentera hrefhttpssyliuscomplugins targetblankimg srchttpssyliuscomassetsbadgeofficialsyliuspluginpng width200ap\n\np aligncenterthis plugin allows customers to cancel the placed order before it is processedp\n\nscreenshot showing the customers orders page with cancel buttonsdocsscreenshotpng\n\n business value\n\nso far once a customer changed their mind about already placed order it was up to the administrator to cancel the order\nhowever we have asked ourselves a question  why cant customer cancel the order when it is yet to be

In [131]:
words_clean = remove_url(words)

In [132]:
words_clean[0:1000]

' Customer Analytics iPython Notebook See httpkpei meblogp  warning BEWARE  This repository has been deprecated and will not be maintained or evolved by the Sylius Team You can still use it with compatible Sylius versions but at your own risk as no bugs will be fixed on it p aligncenter a hrefhttpssylius com targetblank img srchttpsdemo sylius comassetsshopimglogo png  a p h aligncenterCustomer Order Cancellation Pluginh p aligncentera hrefhttpssylius complugins targetblankimg srchttpssylius comassetsbadgeofficialsyliusplugin png widthap p aligncenterThis plugin allows customers to cancel the placed order before it is processed p Screenshot showing the customers orders page with cancel buttonsdocsscreenshot png  Business Value So far once a Customer changed their mind about already placed Order it was up to the Administrator to cancel the order However we have asked ourselves a question  why cant Customer cancel the order when it is yet to be paid Here comes Customer Order Cancellation

In [83]:
type(words_clean)

str

In [84]:
frequencies = pd.Series(words_clean.split()).value_counts()

In [85]:
frequencies.head()

customer     593
data         404
customers    262
model        184
1            181
dtype: int64

In [37]:
def remove_stopwords():
    '''
    This function removes English language stop words from our texts.
    '''
    stopword_list = stopwords.words('english')

    stopword_list.remove('no')
    stopword_list.remove('not')
    
    words = content.split()
    filtered_words = [w for w in words if w not in stopword_list]
    
    content_without_stopwords = ' '.join(filtered_words)
    
    print(content_without_stopwords)

In [34]:
words_clean = clean(words)

# Customer Analytics iPython Notebook See http://kpei.me/blog/?p=921 > :warning: **BEWARE!** > This repository deprecated not maintained evolved Sylius Team. You still use compatible Sylius versions, risk, no bugs fixed it. <p align="center"> <a href="https://sylius.com" target="_blank"> <img src="https://demo.sylius.com/assets/shop/img/logo.png" /> </a> </p> <h1 align="center">Customer Order Cancellation Plugin</h1> <p align="center"><a href="https://sylius.com/plugins/" target="_blank"><img src="https://sylius.com/assets/badge-official-sylius-plugin.png" width="200"></a></p> <p align="center">This plugin allows customers cancel placed order processed.</p> ![Screenshot showing customer's orders page cancel buttons](docs/screenshot.png) ## Business Value So far, Customer changed mind already placed Order, Administrator cancel order. However, asked question - can't Customer cancel order yet paid? Here comes Customer Order Cancellation Plugin allows canceling unpaid order straight order 

In [35]:
frequencies = pd.Series(words_clean.split()).value_counts()

AttributeError: 'NoneType' object has no attribute 'split'